# First look at the Santa 2023 Problem

This notebook was created during the twitch/youtube stream you can watch here:

https://www.youtube.com/watch?v=hNLJwpyN9iE

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sympy.combinatorics import Permutation
from pprint import pprint

## Understanding the Data

In [2]:
info = pd.read_csv('puzzle_info.csv')
puzzles = pd.read_csv('puzzles.csv')
ss = pd.read_csv('sample_submission.csv')

In [12]:
example = puzzles.loc[1]

sol = example['solution_state']
init = example['initial_state']

moves = info.loc[info['puzzle_type'] == example['puzzle_type']]['allowed_moves'].values[0]
moves = eval(moves)

# Convert to numpy array
init = np.array(init.split(';'))
sol = np.array(sol.split(';'))

# Add inverse moves and convert to permutations.
moves_ = moves.copy()
for m, d in moves_.items():
        moves[f'-{m}'] = Permutation(d) ** -1
        moves[m] = Permutation(d) 
        
moves

{'f0': Permutation(23)(2, 19, 21, 8)(3, 17, 20, 10)(4, 6, 7, 5),
 'f1': Permutation(0, 18, 23, 9)(1, 16, 22, 11)(12, 13, 15, 14),
 'r0': Permutation(1, 5, 21, 14)(3, 7, 23, 12)(8, 10, 11, 9),
 'r1': Permutation(23)(0, 4, 20, 15)(2, 6, 22, 13)(16, 17, 19, 18),
 'd0': Permutation(6, 18, 14, 10)(7, 19, 15, 11)(20, 22, 23, 21),
 'd1': Permutation(23)(0, 1, 3, 2)(4, 16, 12, 8)(5, 17, 13, 9),
 '-f0': Permutation(23)(2, 8, 21, 19)(3, 10, 20, 17)(4, 5, 7, 6),
 '-f1': Permutation(0, 9, 23, 18)(1, 11, 22, 16)(12, 14, 15, 13),
 '-r0': Permutation(1, 14, 21, 5)(3, 12, 23, 7)(8, 9, 11, 10),
 '-r1': Permutation(23)(0, 15, 20, 4)(2, 13, 22, 6)(16, 18, 19, 17),
 '-d0': Permutation(6, 10, 14, 18)(7, 11, 15, 19)(20, 21, 23, 22),
 '-d1': Permutation(23)(0, 2, 3, 1)(4, 8, 12, 16)(5, 9, 13, 17)}

# Solver

In [4]:
def apply_move(solution,state):
    for move in solution:
        state=moves[move](state)
    return state

In [5]:
# Sample solution if you want
ss_moves = ss.loc[0]['moves'].split('.')
ss_moves

['r1', '-f1']

In [6]:
cube = init.copy()
cube = apply_move(ss_moves,cube)
cube

['A',
 'A',
 'A',
 'A',
 'B',
 'B',
 'B',
 'B',
 'C',
 'C',
 'C',
 'C',
 'D',
 'D',
 'D',
 'D',
 'E',
 'E',
 'E',
 'E',
 'F',
 'F',
 'F',
 'F']

In [59]:
def apply_move(solution,state):
    for move in solution:
        state=moves[move](state)
    return state

for i , d in puzzles.query('puzzle_type == "cube_2/2/2"').iterrows():
    init = np.array(d['initial_state'].split(';'))
    ss_moves = ss.loc[ss['id'] == d['id']]['moves'].values[0]
    ss_moves = ss_moves.split('.')
    solved_cube = apply_move(ss_moves,init)
    print(f"Id {d['id']}: Solved state {solved_cube}")
    assert ';'.join(solved_cube) == d['solution_state']

Id 0: Solved state ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F']
Id 1: Solved state ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F']
Id 2: Solved state ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F']
Id 3: Solved state ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F']
Id 4: Solved state ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F']
Id 5: Solved state ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F']
Id 6: Solved state ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F']
Id 7: Solved state [

# Brute Force Optimization

In [ ]:
from queue import Queue

def find_solution(initial_state, solution_state, moves):
    # Function to check if the state matches the solution state
    def is_solution(state):
        return np.array_equal(state, solution_state)

    # Start with the initial state
    q = Queue()
    q.put((initial_state, []))
    visited = set()

    while not q.empty():
        current_state, current_solution = q.get()

        if is_solution(current_state):
            return current_solution

        for move_name, move in moves.items():
            new_state = move(current_state.copy())
            new_state_tuple = tuple(new_state)

            if new_state_tuple not in visited:
                visited.add(new_state_tuple)
                q.put((new_state, current_solution + [move_name]))

    return None  # Return None if no solution is found

# For 2x2 cubes

In [82]:
solutions={}

In [91]:
for id,row in puzzles.loc[3:29].iterrows():
    puzzle_id=id
    initial_state = np.array(row.initial_state.split(';'))
    solution_state = np.array(row.solution_state.split(';'))
    moves = moves 

    

    solution_moves = find_solution(initial_state, solution_state, moves)
    print(f"'Solution id: {id}, Solution moves:{solution_moves}")
    solutions[id]= solution_moves


    

'Solution id: 3, Solution moves:['f1', 'r0', 'f1', 'r0', 'd1', 'f1', 'r0', 'd1', 'r1', '-f0']


In [85]:
solutions

{0: ['r1', '-f1'],
 1: ['f0', 'r1', 'f1', 'd0', 'd0', 'f1', 'd1', '-r1', '-d1'],
 2: ['f0',
  'f0',
  'r0',
  'r0',
  'f1',
  'd0',
  '-f1',
  '-d1',
  '-f0',
  '-d0',
  '-f1',
  '-r1']}

In [79]:
solution_state

array(['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D',
       'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F'], dtype='<U1')

In [69]:
puzzle_id

1

In [45]:
solutions

{397: ['f0', 'r1', 'f1', 'd0', 'd0', 'f1', 'd1', '-r1', '-d1']}

In [21]:
puzzles

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0
...,...,...,...,...,...
393,393,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0
394,394,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0
395,395,globe_3/33,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0
396,396,globe_8/25,A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...,V;P;F;L;P;X;O;A;J;b;V;Y;D;Y;C;X;J;F;U;G;d;L;b;...,0


In [11]:
sol

array(['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C', 'D',
       'D', 'D', 'D', 'E', 'E', 'E', 'E', 'F', 'F', 'F', 'F'], dtype='<U1')

In [10]:
puzzles

,id,puzzle_type,solution_state,initial_state,num_wildcards
0,0,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C,0
1,1,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,D;E;C;B;B;E;F;A;F;D;B;F;F;E;B;D;A;A;C;D;C;E;A;C,0
2,2,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;F;C;C;F;A;D;D;B;B;A;F;E;B;C;A;A;B;D;F;E;E;C;D,0
3,3,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,A;C;E;C;F;D;E;D;A;A;F;A;B;D;B;F;E;D;B;F;B;C;C;E,0
4,4,cube_2/2/2,A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F,E;D;E;D;A;E;F;B;A;C;F;D;F;D;C;A;F;B;C;C;B;E;B;A,0
...,...,...,...,...,...
393,393,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,D;D;L;A;P;E;R;U;U;C;S;R;J;B;E;G;O;J;F;Q;R;E;D;...,0
394,394,globe_3/33,A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...,V;L;N;G;B;V;R;E;H;A;K;S;I;N;G;E;V;C;L;G;S;M;P;...,0
395,395,globe_3/33,N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...,N12;N219;N227;N198;N4;N208;N214;N245;N56;N55;N...,0
396,396,globe_8/25,A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...,V;P;F;L;P;X;O;A;J;b;V;Y;D;Y;C;X;J;F;U;G;d;L;b;...,0


In [19]:
ss.iloc[1].moves.split('.')

['f1',
 'd0',
 '-r0',
 '-f1',
 '-d0',
 '-f1',
 'd0',
 '-r0',
 'f0',
 '-f1',
 '-r0',
 'f1',
 '-d1',
 '-r1',
 'f1',
 'd1',
 'r0',
 'f1',
 'd1',
 'd0',
 '-r0',
 '-d0',
 '-d0',
 'f1',
 'd1',
 'd0',
 '-r0',
 '-d0',
 '-r0',
 '-d0',
 '-d0',
 'r0',
 '-d0',
 '-f1',
 'd0',
 '-r0',
 '-r1',
 'f1',
 '-f0',
 'f1',
 '-d1',
 'd0',
 '-r0',
 '-d0',
 '-d0',
 'f1',
 '-d1',
 'r0',
 '-d0',
 '-d1',
 '-f1',
 '-r1',
 'f1',
 'd1',
 '-d0',
 '-f1',
 '-d0',
 '-d1',
 'r0',
 '-d0',
 '-d1',
 '-f1',
 '-f0']